In [1]:
api_key = "wnj7Fn4IKO4n24VxCiJS6XZoW"
api_key_secret = "p48mEv3kxOvUis0GekcSBFaLm8bAxA355L0QTPmkIRTHfYPDpf"
access_token = "1217079490922213378-niOQ2I85oktyg5GxepYBMXUJGFweWi"
access_token_secret = "IsIq9JrI0bbHU4jNLtdhsg1lKXqZcZOg7WpCbvgD08ICD"

In [2]:
import tweepy
import re
import pandas as pd
import string
import nltk
import random
from nltk.tokenize import word_tokenize

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

In [5]:
api = tweepy.API(auth)

In [6]:
cursor = tweepy.Cursor(api.user_timeline, id='MichaelVaughan', tweet_mode='extended').items(10000)

In [7]:
#saving these texts
texts = []
for i in cursor:
    line = i.full_text
    line = line.strip()
    texts.append(re.sub('\n', '', line))

In [8]:
print(texts)

['Well that was a mad day of Batting but very entertaining … #ENGvIND', 'That’s just the way I play !!!!!!!!!!! Sorry but that was rubbish …. but England have bowled very very well today …. #ENGvIND', 'The non selection of @ashwinravi99 has to be greatest NON selection we have ever witnessed across 4 Tests in the UK !!! 413 Test wickets &amp; 5 Test 100s !!!! #ENGvIND Madness …', 'Time to get down to the Oval !! Weather set fair for the week … England stronger this week I reckon .. India also stronger if @ashwinravi99 plays .. England to win though .. This tour has a 2014 feel to it .. #ENGvIND', 'RT @GroundWOW: Activate and monetise ground real estate. Full colour brand activations. Book your demo today. info@groundwow.com #groundpri…', 'Peter … Why is it Odd ?? James isn’t in the England team so he is free … he hasn’t been as many bio bubbles as England players … England will send a team but not sure what it will look like at this stage … https://t.co/rjoHBBhh1M', 'What a great caree

In [9]:
# hi = 0
# en = 0

In [10]:
# for i in texts:
#     blob = TextBlob(i)
#     if blob.detect_language() == 'en':
#         en += 1
#     elif blob.detect_language() == 'hi':
#         hi += 1

In [11]:
def clean_txt(txt):
    cleaned_txt = []
    url = re.compile(r'https?://\S+|www\.S+')
    html = re.compile(r'<.*?>')
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    punct_table = str.maketrans('', '', string.punctuation)
    for line in txt:
        line = line.lower()
        line = url.sub(r'', line)
        line = html.sub(r'', line)
        line = emoji_pattern.sub(r'', line)
        line = line.translate(punct_table)
        if len(line) >= 3:
            tokens = word_tokenize(line)
            words = [word for word in tokens if word.isalpha()]
            cleaned_txt += words
    return cleaned_txt

In [12]:
txt = clean_txt(texts)

In [13]:
txt

['well',
 'that',
 'was',
 'a',
 'mad',
 'day',
 'of',
 'batting',
 'but',
 'very',
 'entertaining',
 'engvind',
 'that',
 's',
 'just',
 'the',
 'way',
 'i',
 'play',
 'sorry',
 'but',
 'that',
 'was',
 'rubbish',
 'but',
 'england',
 'have',
 'bowled',
 'very',
 'very',
 'well',
 'today',
 'engvind',
 'the',
 'non',
 'selection',
 'of',
 'has',
 'to',
 'be',
 'greatest',
 'non',
 'selection',
 'we',
 'have',
 'ever',
 'witnessed',
 'across',
 'tests',
 'in',
 'the',
 'uk',
 'test',
 'wickets',
 'amp',
 'test',
 'engvind',
 'madness',
 'time',
 'to',
 'get',
 'down',
 'to',
 'the',
 'oval',
 'weather',
 'set',
 'fair',
 'for',
 'the',
 'week',
 'england',
 'stronger',
 'this',
 'week',
 'i',
 'reckon',
 'india',
 'also',
 'stronger',
 'if',
 'plays',
 'england',
 'to',
 'win',
 'though',
 'this',
 'tour',
 'has',
 'a',
 'feel',
 'to',
 'it',
 'engvind',
 'rt',
 'groundwow',
 'activate',
 'and',
 'monetise',
 'ground',
 'real',
 'estate',
 'full',
 'colour',
 'brand',
 'activations',
 

In [14]:
def create_markov_dict(txt, n_gram=2):
    markov_dict = {}
    for i in range(len(txt) - n_gram - 1):
        curr_state, next_state = "", ""
        for j in range(n_gram):
            curr_state += txt[i + j] + " "
            next_state += txt[i + j + n_gram] + " "
        curr_state = curr_state.strip()
        next_state = next_state.strip()
        if curr_state not in markov_dict:
            markov_dict[curr_state] = {}
            markov_dict[curr_state][next_state] = 1
        else:
            if next_state in markov_dict[curr_state]:
                markov_dict[curr_state][next_state] += 1
            else:
                markov_dict[curr_state][next_state] = 1
    
    for curr_state, transition in markov_dict.items():
        total = sum(transition.values())
        for state, count in transition.items():
            markov_dict[curr_state][state] = count/total
    
    return markov_dict
        
def generate_tweet(markov_dict, begin_word, length=100):
    n = 0
    curr_state = begin_word
    next_state = None
    generated_tweet = ""
    generated_tweet += curr_state + " "
    while n < length:
        next_state = random.choices(
            list(markov_dict[curr_state].keys()), 
            list(markov_dict[curr_state].values())
        )
        curr_state = next_state[0]
        generated_tweet += curr_state + " "
        n += 1
    return generated_tweet

In [15]:
model = create_markov_dict(txt, 2)
print(model.keys())

dict_keys(['well that', 'that was', 'was a', 'a mad', 'mad day', 'day of', 'of batting', 'batting but', 'but very', 'very entertaining', 'entertaining engvind', 'engvind that', 'that s', 's just', 'just the', 'the way', 'way i', 'i play', 'play sorry', 'sorry but', 'but that', 'was rubbish', 'rubbish but', 'but england', 'england have', 'have bowled', 'bowled very', 'very very', 'very well', 'well today', 'today engvind', 'engvind the', 'the non', 'non selection', 'selection of', 'of has', 'has to', 'to be', 'be greatest', 'greatest non', 'selection we', 'we have', 'have ever', 'ever witnessed', 'witnessed across', 'across tests', 'tests in', 'in the', 'the uk', 'uk test', 'test wickets', 'wickets amp', 'amp test', 'test engvind', 'engvind madness', 'madness time', 'time to', 'to get', 'get down', 'down to', 'to the', 'the oval', 'oval weather', 'weather set', 'set fair', 'fair for', 'for the', 'the week', 'week england', 'england stronger', 'stronger this', 'this week', 'week i', 'i r

In [1]:
print(generate_tweet(markov_dict = model, begin_word = "on cricbuzzchatter", length = 15))

NameError: name 'generate_tweet' is not defined